# Random Forest Regressor...

### Python Library ...

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense,Dropout
from tensorflow.keras.optimizers import Adam,SGD
from matplotlib import pyplot as plt
from imblearn.over_sampling import SMOTE

### Load All The Data....

In [2]:
# Load the data
file_path = 'All_Data.xlsx'  # Replace with your file path
data_01 = pd.read_excel(file_path)

### Creating Synthetic Data ....

In [3]:
import numpy as np
from sklearn.neighbors import NearestNeighbors

# Original dataset (features only)
# X = np.array([[1, 2], [2, 3], [3, 4], [5, 6]])
Data = data_01.loc[:, 'Average ROM Angle':90].values 

# Number of synthetic samples to generate
num_synthetic_samples = 100

# Fit KNN to find nearest neighbours
knn = NearestNeighbors(n_neighbors=2)  # Use 2 neighbours for simplicity
knn.fit(Data)

# Generate synthetic samples
synthetic_samples = []
for _ in range(num_synthetic_samples):
    # Randomly pick a data point from X
    idx = np.random.randint(0, Data.shape[0])
    point = Data[idx]

    # Find neighbours of the point
    neighbours = knn.kneighbors([point], return_distance=False)[0]
    
    # Randomly choose one neighbour
    neighbour = Data[neighbours[np.random.randint(1, len(neighbours))]]  # Skip self (idx=0)
    
    # Interpolate between the point and the neighbour
    alpha = np.random.uniform(0, 1)  # Random weight for interpolation
    synthetic_point = (point + alpha * (neighbour - point))
    
    synthetic_samples.append(synthetic_point)

# Combine original and synthetic samples
synthetic_samples = np.array(synthetic_samples)
augmented_dataset = np.vstack([Data, synthetic_samples])

data = pd.DataFrame(augmented_dataset)


### Extract EEG features and target...

In [4]:
# Extract EEG features and target
X = data.loc[:, 1:181].values  # EEG data (columns D to MI) iloc[:,3:]
y = data.loc[:, 0].values  # Target ROM angle (column C)
max_rom = y.max()
min_rom = y.min()
# Check for extreme values in target
print(f"Max ROM Angle: {max_rom}, Min ROM Angle: {min_rom}")

# Normalise the target variable
y = (y - min_rom) / (max_rom - min_rom)


Max ROM Angle: 155.0, Min ROM Angle: 105.0


### Normalise the features...

In [5]:
# Normalise the features
scaler = MinMaxScaler()
X_normalised = scaler.fit_transform(X)

# Reshape X for GRU (samples, timesteps, features)
n_features = X_normalised.shape[1]
X_reshaped = X_normalised.reshape(X_normalised.shape[0], 1, n_features)

In [6]:
X_reshaped.shape

(127, 1, 181)

In [7]:
reshaped_array = X_reshaped.squeeze(axis=1)  # Remove the second dimension
print(reshaped_array.shape)


(127, 181)


### Split data into training and testing sets ...

In [8]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y, test_size=0.1, random_state= 4)

In [9]:
X_test.shape

(13, 1, 181)

In [10]:
import numpy as np

# Reshape to 2D
X_X = X_train.squeeze(axis=1)
X_Xtest = X_test.squeeze(axis=1)

print(X_X)

[[0.29721114 0.5397973  0.54256638 ... 0.         0.         0.        ]
 [0.34401183 0.03094984 0.         ... 0.07936508 0.07277167 0.02661783]
 [0.8336619  0.80576307 0.82462892 ... 0.         0.         0.        ]
 ...
 [0.89635201 0.95291565 0.99539845 ... 0.         0.         0.        ]
 [0.51975886 0.55804037 0.52439283 ... 0.11885717 0.11762282 0.12261164]
 [0.265252   0.19260634 0.16826609 ... 0.         0.         0.        ]]


### SVR Model..

In [16]:
from sklearn.svm import SVR

model = SVR(kernel='rbf', C=1.0, epsilon=0.1)
model.fit(X_X, y_train)
predictions_svr = model.predict(X_Xtest)


### Final R2 Score...

In [21]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Make predictions
predictions_svr = model.predict(X_Xtest)

r2 = r2_score(y_test,predictions_svr)
mae = mean_absolute_error(y_test,predictions_svr)
mse = mean_squared_error(y_test,predictions_svr)

print(f'R2 score: {r2*100:2f}%')
print(f'MAE : {mae:5f}')
print(f'MSE : {mse:5f}')

R2 score: 69.505714%
MAE : 0.077115
MSE : 0.007842


### Invers The Metrix...

In [25]:
# Function for invers metrixes...
def invers_prediction(predic):
    prz = predic
    Predictions = []
    for i in range(prz.shape[0]):
        pred = prz[i]*(max_rom-min_rom) + min_rom
        Predictions.append(pred)

    return Predictions


In [27]:
inv_predicted = invers_prediction(predictions_svr)
# prediction_list = [float(arr[0]) for arr in inv_predicted] 
# predicteded_list = [[value] for value in prediction_list]
# prediction_list
inver_y_test = invers_prediction(y_test)


### R2 Score for Invers metrixes...

In [31]:
r2 = r2_score(inver_y_test,inv_predicted)
mae = mean_absolute_error(inver_y_test,inv_predicted)
mse = mean_squared_error(inver_y_test,inv_predicted)

print(f'R2 score: {r2*100:2f}%')
print(f'MAE : {mae:5f}')
print(f'MSE : {mse:5f}')

R2 score: 69.505714%
MAE : 3.855726
MSE : 19.605668


In [32]:
pd_result_table = pd.DataFrame(data = [inver_y_test,inv_predicted], index = ["y-test","prd"])
pd_result_table

,0,1,2,3,4,5,6,7,8,9,10,11,12
y-test,127.737941,134.600000,146.660000,131.700000,136.940497,135.000000,140.637596,135.600000,131.138443,121.6000,141.088760,124.584759,116.724355
prd,123.638908,134.317671,142.143379,138.705222,140.201370,128.344081,141.315602,131.205863,129.882109,127.8635,135.962871,122.878951,121.605125
